# Configuration and Infrastructure

In [1]:
import os

data_dpath = os.path.join(os.getcwd(), "data")
os.chdir(os.path.pardir)

In [2]:
MINUTE = 60
HOUR = 60 * MINUTE

FRAME_RESOLUTIONS = [ MINUTE, 5 * MINUTE, 10 * MINUTE, 30 * MINUTE, HOUR ]
BUFFER_CAPACITY = 50
CONTAINER_CAPACITY = 2000

## Making Candle Buffers

In [3]:
from src.candle_buffer import CandleBuffer

candles_buffers = {
    frame_resolution: CandleBuffer(frame_resolution, BUFFER_CAPACITY, CONTAINER_CAPACITY)
    for frame_resolution in FRAME_RESOLUTIONS
}

## Making Indicators

In [4]:
from src.indicator import FeaturesCompiler
from src.indicator.fear_and_greed import CryptoFearAndGreed
from src.indicator.momentum import get_momentum_indicator_suite
from src.indicator.pattern import get_pattern_indicator_suite

indicators = [ CryptoFearAndGreed(apply_scaling=True), ]

for candle_buffer in candles_buffers.values():
    indicators.extend(get_momentum_indicator_suite(candle_buffer))
    indicators.extend(get_pattern_indicator_suite(candle_buffer))

features_compiler = FeaturesCompiler(indicators)
features_compiler.get_num_features()

471

## Making and Saving Data Pipeline Image

In [5]:
data_pipeline = {
    "candle_buffers": candles_buffers,
    "features_compiler": features_compiler
}

In [6]:
import pickle

with open(os.path.join(data_dpath, "data_pipeline.pickle"), "wb") as file:
    pickle.dump(data_pipeline, file)

## Loading Data Pipeline Image

In [22]:
with open(os.path.join(data_dpath, "data_pipeline.pickle"), "rb") as file:
    data_pipeline = pickle.load(file)
    
    candle_buffers = data_pipeline.get("candle_buffers")
    features_compiler = data_pipeline.get("features_compiler")